## Experimental Workflows


In [ ]:
%load_ext autoreload
%autoreload 2

from fibsem import utils

microscope, settings = utils.setup_session()

### Automated Quality Control

In [ ]:

# do contrast test polishing
protocol = {
    "milling": {
        "polish": {
            "stages": [{
        "application_file": "autolamella",
        "cross_section": "CleaningCrossSection",
        "hfw": 40e-6,
        "height": 6.0e-07,
        "width": 6.0e-06,
        "depth": 4.0e-07,
        "milling_current": 6.0e-11,
        "milling_voltage": 3.0e3,
        "type": "Rectangle",
        "name": "AdaptivePolishing",
            }
        ],
        "point": {
            "x": 0.0,
            "y": 5e-6,}
        }
    },
    "options": {
        "experimental": {
            "adaptive_polishing": {
                "threshold": 100,
                "step_size": 5e-6,
                "step_limit": 10,
                "image_resolution": [3072, 2188],
                "image_line_integration": 20,
                "image_dwell_time": 100e-9,
            }

    }
}
}

from autolamella.workflows.experimental import adaptive_mill_polishing


adaptive_mill_polishing(microscope, settings, protocol)

## Parallel Trench Milling

REF: https://www.biorxiv.org/content/10.1101/2024.06.20.599830v1.full.pdf

In [ ]:
%load_ext autoreload
%autoreload 2

import os

import matplotlib.pyplot as plt
from fibsem import utils, conversions, patterning, milling, acquire
from fibsem import config as fcfg
from fibsem.imaging import _tile
from fibsem.patterns.ui import draw_milling_patterns
from fibsem.structures import ImageSettings, BeamType, FibsemImage, Point
from autolamella.structures import Experiment, AutoLamellaWaffleStage
from autolamella.workflows.core import start_of_stage_update, end_of_stage_update
# Steps:
# 1. Open AutoLamella, create a new experiment, load waffle protocol
# 2. Open Minimap, acquire Ion overview (max size 960um), select positions
# 3. Run this script to parallelize the trench milling

# paths
PATH = r"C:/Users/Admin/Github/autolamella/autolamella/log\DEV-PARALLEL-TRENCH"
EXPERIMENT_PATH = os.path.join(PATH, "experiment.yaml")
PROTOCOL_PATH = os.path.join(PATH, "protocol.yaml")

microscope, settings = utils.setup_session(protocol_path=PROTOCOL_PATH)

# load the experiment
exp = Experiment.load(os.path.join(PATH, "experiment.yaml"))

# load the overview image
overview_image = FibsemImage.load(os.path.join(PATH, "overview-image.tif"))
# TODO: we should take this image instead of loading it

# move to the overview image position
microscope.move_stage_absolute(overview_image.metadata.microscope_state.stage_position)

# get the positions from the experiment (trench positions)
positions = [position for position in exp.positions if position.state.stage is AutoLamellaWaffleStage.ReadyTrench]
stage_positions = [pos.state.microscope_state.stage_position for pos in positions]
points = _tile._reproject_positions(overview_image, stage_positions)

# convert the points to microscope image coordinates
milling_points = []
for point in points:
    mp = conversions.image_to_microscope_image_coordinates(point, overview_image.data, overview_image.metadata.pixel_size.x)
    mp.name = point.name
    milling_points.append(mp)

# check the points are within the field of view
# get the maximum field of view
shape = overview_image.data.shape
MAX_HFW = 960e-6 # ION BEAM MAX HFW
MAX_VFW = MAX_HFW * (shape[0] / shape[1])
print("Maximum FOV: ", MAX_HFW, MAX_VFW)

# check if the points are outside the field of view

for point, mp in zip(points, milling_points):
    if mp.x > MAX_HFW or mp.x < -MAX_HFW:
        print(f"Point {point.name} is outside the field of view")
    if mp.y > MAX_VFW or mp.y < -MAX_VFW:
        print(f"Point {point.name} is outside the field of view")

# plot the stage positions on the overview image
plt.imshow(overview_image.data, cmap="gray")
for point, pos in zip(points, stage_positions):
    plt.plot(point[0], point[1], "ro",) #label=pos.name)
plt.legend()

plt.show()


# ASSUMPTION: 
# all trenchs fit within a single max fov

# IMPROVEMENTS:
# convert the milling points into the proper protocol format for consistency
# Implement in the user interface
# Support for selecting outside max fov
# Support for logging statistics, start/end stage updates

In [ ]:

print('MILLING POINTS')


base_position = overview_image.metadata.microscope_state.stage_position
ov_hfw = overview_image.metadata.image_settings.hfw
ov_resolution = overview_image.metadata.image_settings.resolution
microscope.move_stage_absolute(base_position)


settings.image.beam_type = BeamType.ION
settings.image.hfw = 800e-6
settings.image.resolution = ov_resolution
ov_image = acquire.acquire_image(microscope, settings.image)

milling_stages = []
for pos, mp, sp in zip(positions, milling_points, stage_positions):

    # tmp simplified
    pos.protocol["trench"]["depth"] = 0.2e-6

    stages = patterning.get_milling_stages("trench", pos.protocol, point=mp)

    stages[0].name = f"Trench {mp.name}"
    stages[0].milling.hfw = ov_hfw

    milling_stages.extend(stages)



fig = draw_milling_patterns(ov_image, milling_stages)
plt.show()

# TODO: image before milling to correctly display hfw on ui
# TODO: fix. this does not mill them all as the same 'stage', it mills them as separate stages so the hfw is wrong? set hfw in init loop?

In [ ]:
# mill the stages
milling.mill_stages(microscope, milling_stages)

In [ ]:

# mark each position as finished trench milling
# go to each trench position and acquire reference images, mark finished? basically do trench workflow just dont mill



for pos in positions:
    
    # book keeping
    # note: the trench milling duration will not be correct if we bookkeep like this?
    pos = start_of_stage_update(
                microscope,
                pos,
                next_stage=AutoLamellaWaffleStage.MillTrench, 
                parent_ui=None,
            )
    
    settings.image.path = pos.path

    # move to the position
    microscope.move_stage_absolute(pos.state.microscope_state.stage_position)

    # acquire reference images
    reference_images = acquire.take_set_of_reference_images(
        microscope=microscope,
        image_settings=settings.image,
        hfws=[fcfg.REFERENCE_HFW_MEDIUM, fcfg.REFERENCE_HFW_HIGH],
        filename=f"ref_{pos.state.stage.name}_final",
    )

    # book keeping
    exp = end_of_stage_update(microscope, exp, pos, None)

